<a href="https://colab.research.google.com/github/chrishendra93/MI_Workshop/blob/main/Mnist_Multiple_Instance_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/chrishendra93/MI_Workshop


Cloning into 'MI_Workshop'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 1), reused 10 (delta 1), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
np.random.seed(0)

In [ ]:
root_dir = "./MI_Workshop/mnist_clr"

mnist_columns = ["label"] + ["features_{}".format(i) for i in range(28 ** 2)]
mnist_train = pd.read_csv("./sample_data/mnist_train_small.csv", names=mnist_columns)
mnist_test = pd.read_csv("./sample_data/mnist_test.csv", names=mnist_columns)
mnist_arr_train = mnist_train[["features_{}".format(i) for i in range(28 ** 2)]].values
mnist_arr_test = mnist_test[["features_{}".format(i) for i in range(28 ** 2)]].values

X_train = np.load(os.path.join(root_dir, "train_features.npy"))
X_test = np.load(os.path.join(root_dir, "test_features.npy"))
y_train = np.load(os.path.join(root_dir, "train_labels.npy"))
y_test = np.load(os.path.join(root_dir, "test_labels.npy"))

print(np.all(y_train == mnist_train["label"].values))
print(np.all(y_test == mnist_test["label"].values))

train_bags, train_labels = np.load(os.path.join(root_dir, "train_bags.npy")), np.load(os.path.join(root_dir, "train_bags_labels.npy"))
test_bags, test_labels = np.load(os.path.join(root_dir, "test_bags.npy")), np.load(os.path.join(root_dir, "test_bags_labels.npy"))

True
True


In [ ]:
mnist_train.head()

,label,features_0,features_1,features_2,features_3,features_4,features_5,features_6,features_7,features_8,features_9,features_10,features_11,features_12,features_13,features_14,features_15,features_16,features_17,features_18,features_19,features_20,features_21,features_22,features_23,features_24,features_25,features_26,features_27,features_28,features_29,features_30,features_31,features_32,features_33,features_34,features_35,features_36,features_37,features_38,...,features_744,features_745,features_746,features_747,features_748,features_749,features_750,features_751,features_752,features_753,features_754,features_755,features_756,features_757,features_758,features_759,features_760,features_761,features_762,features_763,features_764,features_765,features_766,features_767,features_768,features_769,features_770,features_771,features_772,features_773,features_774,features_775,features_776,features_777,features_778,features_779,features_780,features_781,features_782,features_783
0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Now that our features are aligned, let's visualize some images

In [ ]:
# ENTER CODE TO VISUALIZE THE IMAGE


In [ ]:
# ENTER CODE TO VISUALIZE distribution of bag labels and the number of bags in both train and test set

In [ ]:
# ENTER CODE TO VISUALIZE distribution of bag size 

Distribution of labels seems to be equal between the two bags, let's check the distribution of instances

Let's visualize some of these bag contents

In [ ]:
def visualize_bags(mnist_arr, bags, n_bags):
  sampled_indices = np.random.choice(len(bags), n_bags, replace=False)
  sampled_bags = np.array(bags)[sampled_indices]
  max_instances_num = np.max([len(bag[0]) for bag in sampled_bags])
  _, axes = plt.subplots(len(sampled_bags), max_instances_num,
                         figsize=(10 * len(sampled_bags), 
                                  10 * max_instances_num))
  for idx, bag in zip(np.arange(n_bags), sampled_bags):
    ax = axes[idx, :]
    bag_label = bag[-1]
    instance_size = None
    for i in range(max_instances_num):
      if i >= len(bag[0]):
        ax[i].imshow(np.zeros(instance_size)) # Pad with empty images if bag has fewer instances than max
      else:
        img = mnist_arr[bag[0][i]]
        if instance_size is None:
          w = int(np.sqrt(len(img)))
          instance_size = (w, w)
        ax[i].imshow(img.reshape(instance_size))
        if i == max_instances_num // 2:
          ax[i].set_title('Bag Label: {}'.format(bag_label), fontsize=50)


In [ ]:
# visualize_bags(mnist_arr_train, train_bags, 5)

In [ ]:
def extract_summary_stats(mnist_features, bags):
  max_ = np.array([np.max(mnist_features[bag[0]]) for bag in bags])
  min_ = np.array([np.min(mnist_features[bag[0]]) for bag in bags])
  mean_ = np.array([np.mean(mnist_features[bag[0]]) for bag in bags])
  var_ = np.array([np.var(mnist_features[bag[0]]) for bag in bags])
  med_ = np.array([np.median(mnist_features[bag[0]]) for bag in bags])
  return np.concatenate([max_.reshape(-1, 1), 
                         min_.reshape(-1, 1), 
                         mean_.reshape(-1, 1), 
                         var_.reshape(-1, 1), 
                         med_.reshape(-1, 1)], axis=1)

In [ ]:
X_train_stats = extract_summary_stats(X_train, train_bags)
X_test_stats = extract_summary_stats(X_test, test_bags)

In [ ]:
print(X_train_stats.shape, train_labels.shape)
print(X_test_stats.shape, test_labels.shape)

(4461, 5) (4461,)
(2230, 5) (2230,)


In [ ]:
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# TRANSFORM OBJECT USING PCA and visualize 

In [ ]:
def get_roc_auc(y_true, y_pred):
    fpr, tpr, _  = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    return roc_auc


def get_pr_auc(y_true, y_pred):
    precision, recall, _ = precision_recall_curve(y_true, y_pred, pos_label=1)
    pr_auc = auc(recall, precision)
    return pr_auc


def get_accuracy(y_true, y_pred):
    return balanced_accuracy_score(y_true, y_pred)

def plot_roc_curve(y_true, y_pred, label, ax, sample=False):
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    if sample:
        sns.lineplot(fpr[::sample], tpr[::sample], lw=1, ax=ax, alpha=0.3, label="{} - AUC: %0.2f".format(label) % (roc_auc))
    else:
        sns.lineplot(fpr, tpr, lw=1, ax=ax, alpha=0.3, label="{} - AUC: %0.2f".format(label) % (roc_auc), rasterized=True)

def plot_pr_curve(y_true, y_pred, label, ax, sample=False):
    precision, recall, _ = precision_recall_curve(y_true, y_pred, pos_label=1)
    pr_auc = auc(recall, precision)
    if sample:
        sns.lineplot(recall[::sample], precision[::sample], lw=1, alpha=0.3, ax=ax, label="{} - AUC: %0.2f".format(label) % (pr_auc))
    else:
        sns.lineplot(recall, precision, lw=1, alpha=0.3, ax=ax, label="{} - AUC: %0.2f".format(label) % (pr_auc), rasterized=True)

def customise_roc_pr_plots(axes, label_size=15, font_size=15, linewidth=5, patch_size=20):
    # Customizing plots
    for ax in axes:
        ax.xaxis.label.set_size(label_size)
        ax.yaxis.label.set_size(label_size)
        for axis in [ax.xaxis, ax.yaxis]:
            for tick in axis.get_major_ticks():
                tick.label.set_fontsize(font_size)
        plt.setp(ax.get_legend().get_texts(), fontsize='{}'.format(font_size))
        plt.setp(ax.lines, linewidth=linewidth)
    axes[0].set_xlabel('False Positive Rate')
    axes[0].set_ylabel('True Positive Rate')
    for ax in axes:
        for axis in [ax.xaxis, ax.yaxis]:
            axis.label.set_size(font_size)
    axes[1].set_xlabel('Recall')
    axes[1].set_ylabel('Precision')
    for ax in axes:
        ax.legend(loc='best', prop={'size': patch_size})



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_curve, precision_recall_curve, auc
from scipy.stats import mode

# Train a Logistic Regression Model and visualize results 


Now, le us try to compute Hausdorff distance from the test samples to the train samples

In [ ]:
from sklearn.metrics import pairwise_distances

pairwise_distance = pairwise_distances(X_test, X_train)
print(pairwise_distance.shape)


(10000, 20000)


In [ ]:
all_bag_distances = []

for test_bag in tqdm(test_bags, total=len(test_bags)):
  test_bag_instances = pairwise_distance[test_bag]
  bag_distances = []
  for train_bag in train_bags:
    test_train_distance = np.max(np.min(test_bag_instances[:, train_bag], axis=1))
    train_test_distance = np.max(np.min(test_bag_instances[:, train_bag].transpose(), axis=1))
    bag_distances.append(max(test_train_distance, train_test_distance))
  all_bag_distances.append(np.array(bag_distances).reshape(1, -1))
all_bag_distances = np.concatenate(all_bag_distances, axis=0)
all_bag_distances.shape

(2230, 4461)

In [ ]:
y_pred_test_knn = mode(train_labels[np.argsort(all_bag_distances, axis=1)[:, :10]], axis=1)[0].flatten()
print(accuracy_score(test_labels, y_pred_test_knn))
print(balanced_accuracy_score(test_labels, y_pred_test_knn))

0.6968609865470852
0.6207032139235529


Can we do even better?

In [ ]:
import torch.nn.functional as F
import torch
from torch import nn
from torch.nn import BCELoss
from torch.optim import LBFGS, Adam
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Code Logistic Regression applying Multiple Instance Learning Assumption

In [ ]:
# logit_mi = LogisticRegressionMI(64, mode='max')
# criterion = BCELoss()
# optimizer = LBFGS(logit_mi.parameters(), lr=0.001, max_iter=1000)
# logit_mi.train()


# def closure():
#     optimizer.zero_grad()
#     output = logit_mi(torch.Tensor(X_train), train_bags)
#     loss = criterion(output, torch.Tensor(train_labels))
#     loss.backward()
#     return loss

# optimizer.step(closure) 

# logit_mi.eval()
# with torch.no_grad():
#   y_pred_logit_mi_test = logit_mi(torch.Tensor(X_test), test_bags).detach().numpy()
#   print("ROC AUC: {}".format(get_roc_auc(test_labels, y_pred_logit_mi_test)))
#   print("PR AUC: {}".format(get_pr_auc(test_labels, y_pred_logit_mi_test)))
#   print("Accuracy Score: {}".format(accuracy_score(test_labels, y_pred_logit_mi_test >= 0.5)))
#   print("Balanced Accuracy Score: {}".format(balanced_accuracy_score(test_labels, y_pred_logit_mi_test >= 0.5)))

In [ ]:
# logit_mi = LogisticRegressionMI(64, mode='mean')
# criterion = BCELoss()
# optimizer = LBFGS(logit_mi.parameters(), lr=0.001, max_iter=1000)
# logit_mi.train()


# def closure():
#     optimizer.zero_grad()
#     output = logit_mi(torch.Tensor(X_train), train_bags)
#     loss = criterion(output, torch.Tensor(train_labels))
#     loss.backward()
#     return loss

# optimizer.step(closure) 

# logit_mi.eval()
# with torch.no_grad():
#   y_pred_logit_mi_test = logit_mi(torch.Tensor(X_test), test_bags).detach().numpy()
#   print("ROC AUC: {}".format(get_roc_auc(test_labels, y_pred_logit_mi_test)))
#   print("PR AUC: {}".format(get_pr_auc(test_labels, y_pred_logit_mi_test)))
#   print("Accuracy Score: {}".format(accuracy_score(test_labels, y_pred_logit_mi_test >= 0.5)))
#   print("Accuracy Score: {}".format(balanced_accuracy_score(test_labels, y_pred_logit_mi_test >= 0.5)))